In [7]:
"""
Calculates MFIS estimates for failure probability using ECL
adaptive designs from the Hartmann6 experiment and space-filling designs.

@author: D. Austin Cole  austin.cole8@vt.edu
"""

from datetime import date
import numpy as np
import pandas as pd
from pyDOE import lhs
import scipy.stats as ss
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import os
import sys
fileDir = os.getcwd()
sourcePath = os.path.join(fileDir, 'MFISPy')
sys.path.append(sourcePath)
from mfis import BiasingDistribution
from mfis import MultiFidelityIS
from mfis.mv_independent_distribution import MVIndependentDistribution

sourcePath = fileDir
sys.path.append(sourcePath)
from eclGP import *
import matplotlib.pyplot as plt
import time
import dill
from psis import psislw
import glob
import statistics

In [2]:
os.makedirs("data/Ex_3", exist_ok=True)

In [3]:
Start_time = time.time()
today = date.today()
case = 'Ex3'
Model = Numerical_Ex3()
bounds = ((-5,5),(-5,5),(-5,5),(-5,5),(-5,5),(-5,5),(-5,5),(-5,5),(-5,5),(-5,5))
threshold = 18.74
dim = len(bounds)
alpha = 0.01

N_HIGH_FIDELITY_INPUTS = 1200
n_init = 10*dim
n_select = 600 - n_init
MC_REPS = 25
batch_size = 10

mfis_probs = np.ones((MC_REPS, 3))
#gauss_kernel = 1.0 * RBF(length_scale= np.repeat(0.5, dim),length_scale_bounds=(1e-4, 1e4))
# Stochastic
gauss_kernel = 1.0 * RBF(length_scale= np.repeat(0.5, dim),length_scale_bounds=(1e-4, 1e4)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))



###############################
# 0. Build Initial design
###############################

np.random.seed(1)

stat_results = np.zeros((MC_REPS,dim+1))
initial_designs = np.zeros((n_init, (dim+1)*MC_REPS))
today = date.today()
file_date = today.strftime("%m%d%y")

os.makedirs("data/Ex_3", exist_ok=True)

for i in range(MC_REPS):
    X0 = lhs(dim, n_init)
    X0 = ss.norm.ppf(X0)
    Y0 = Model.predict(X0)

    initial_designs[:,((dim+1)*i):((dim+1)*(i+1))] = \
        np.hstack((X0, Y0.reshape((-1,1))))


np.savetxt(f'data/Ex_3/Initial_designs_{case}_{file_date}.csv',initial_designs, delimiter = ",")

In [4]:
###############################
# 1. Build ECL GP
###############################
initial_designs_df = pd.read_csv(f"data/Ex_3/Initial_designs_{case}_{file_date}.csv",header=None)
initial_designs = np.array(initial_designs_df)
ecl_designs = np.zeros((n_init+n_select, (dim+1)*MC_REPS))
ecl_times = np.zeros((MC_REPS,))
ecl_batch_designs = np.zeros((n_init+n_select, (dim+1)*MC_REPS))
ecl_batch_times = np.zeros((MC_REPS,))

def limit_state_function(y):
    return y - threshold

for i in range(MC_REPS): 
    X0 = initial_designs[:,((dim+1)*i):((dim+1)*(i+1)-1)]
    Y0 = initial_designs[:,(dim+1)*(i+1)-1]
      
    ## Adaptive design with ECL
    ecl_start_time = time.time()
    init_gp = GPR(kernel=gauss_kernel, alpha=1e-6)
    init_gp.fit(X0, Y0)
    eclgp = EntropyContourLocatorGP(init_gp, limit_state_function)
    eclgp.fit(n_select, Model, bounds)

    ecl_designs[:,((dim+1)*i):((dim+1)*(i+1))] = \
        np.hstack((eclgp.X_, eclgp.y_.reshape((-1,1))))
    ecl_times[i] = (time.time() - ecl_start_time)/60
    
#     ## Adaptive design with ECL.batch
#     ecl_batch_start_time = time.time()
#     eclgp_batch = EntropyContourLocatorGP(init_gp,limit_state_function)
    
#     eclgp_batch.fit(np.int(n_select/batch_size), Model, bounds, batch_size=batch_size)    
#     ecl_batch_designs[:,((dim+1)*i):((dim+1)*(i+1))] = \
#         np.hstack((eclgp_batch.X_, eclgp_batch.y_.reshape((-1,1))))
#     ecl_batch_times[i] = (time.time() - ecl_batch_start_time)/60

    print(i)
    np.savetxt(f'data/Ex_3/ecl_designs_{case}_{file_date}.csv', 
                ecl_designs, delimiter = ",")
    np.savetxt(f'data/Ex_3/ecl_designs_times_{case}_{file_date}.csv', 
            ecl_times, delimiter = ",")


/sw/pkgs/arc/python3.11-anaconda/2024.02-1/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/sw/pkgs/arc/python3.11-anaconda/2024.02-1/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 4 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/sw/pkgs/arc/python3.11-anaconda/2024.02-1/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 7 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.


0


In [11]:
####################################
# 2. MFIS
####################################
unif_dist = ss.uniform(0,1)
m1 = 0
s1 = 1
norm_dist = ss.norm(loc=m1, scale=s1)

input_distribution = MVIndependentDistribution(
    distributions=[norm_dist, norm_dist, norm_dist, norm_dist, norm_dist, norm_dist, norm_dist, norm_dist, norm_dist, norm_dist])

## Estimate failure probability (alpha)
'''
num_failures = np.zeros((100,))
for i in range(len(num_failures)):
    new_X = input_distribution.draw_samples(1000000)
    new_Y = hartmann_model.predict(new_X)
    num_failures[i] = np.sum(hartmann_limit_state_function(new_Y)>0)
    print(num_failures[i])
    print(i)
alpha = np.mean(num_failures)/len(new_X)
'''
mfis_probs = np.ones((MC_REPS, 1))

ecl_designs_df = pd.read_csv(f"data/Ex_3/ecl_designs_{case}_{file_date}.csv",header=None) #pd.read_csv(f"data/ecl_designs_{case}_{file_date}.csv",header=None)
ecl_designs = np.array(ecl_designs_df)


weight_is_all_sample_list = []
weight_ucb_all_sample_list = []
    
for j in range(MC_REPS): 
    start_time = time.time()
    ecl_gp = None
       
    ## ECL
    design_X = ecl_designs[:, ((dim+1)*j):((dim+1)*(j+1)-1)]
    design_Y = ecl_designs[:, (dim+1)*(j+1)-1]
    ecl_gp =  GPR(kernel=gauss_kernel, alpha=1e-6)
    ecl_gp.fit(design_X, design_Y)
    
    ## ECL-Batch
#     design_X_batch = ecl_designs_batch[:, ((dim+1)*j):((dim+1)*(j+1)-1)]
#     design_Y_batch = ecl_designs_batch[:, (dim+1)*(j+1)-1]
#     ecl_gp_batch =  GPR(kernel=gauss_kernel, alpha=1e-6)
#     ecl_gp_batch.fit(design_X_batch, design_Y_batch)
            
  
    # Initialize Biasing Distributions
    ecl_bd =  BiasingDistribution(trained_surrogate=ecl_gp,
                            limit_state=limit_state_function,
                            input_distribution=input_distribution)
    ecl_bd_ucb =  BiasingDistribution(trained_surrogate=ecl_gp,
                            limit_state=limit_state_function,
                            input_distribution=input_distribution)
    
    ## Fit Biasing Distributions
    ecl_failed_inputs = np.empty((0, dim))   
    ecl_failed_inputs_ucb = np.empty((0, dim))

    # Get sample outputs from GPs and classify failures  
    for k in range(100):
        sample_inputs = input_distribution.draw_samples(10000) 
        
        ecl_sample_outputs, ecl_sample_std = \
            ecl_gp.predict(sample_inputs, return_std=True)
        ecl_failed_inputs_new = sample_inputs[
            limit_state_function(ecl_sample_outputs.flatten()) > 0,:]
        ecl_failed_inputs = np.vstack((ecl_failed_inputs,
                                       ecl_failed_inputs_new))

        ecl_failed_inputs_ucb_new = sample_inputs[
            limit_state_function(
                ecl_sample_outputs.flatten() + 1.645*ecl_sample_std) > 0,:]
        ecl_failed_inputs_ucb = np.vstack((ecl_failed_inputs_ucb,
                                           ecl_failed_inputs_ucb_new))
        
        if (k % 100) == 0:
            print(k)

        

    print(j)
    # End timer
    end_time = time.time()
    # Calculate elapsed time
    elapsed_time = end_time - start_time
    print(j,'th Experiment',"Total Elapsed time: ", elapsed_time) 
    
    mfis_probs_df = pd.DataFrame(mfis_probs)
    mfis_probs_df = mfis_probs_df.rename(columns={0:'ECL'})
    mfis_probs_df.to_csv(f'data/Ex_3/{case}_mfis_estimates_Experiment_{file_date}_{MC_REPS}.csv',index=False)
dill.dump_session(f'{case}_Stochastic_num_{N_HIGH_FIDELITY_INPUTS}_{MC_REPS}rep')

/sw/pkgs/arc/python3.11-anaconda/2024.02-1/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 3 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/sw/pkgs/arc/python3.11-anaconda/2024.02-1/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


0
0
0 th Experiment Total Elapsed time:  8.563257932662964
